In [28]:
import pandas as pd

# CSV dosyasını oku
data = pd.read_csv("sampled_data.csv")

# Veriye genel bir bakış
print(data.head())


    category                                               text
0  alisveris  A101'den Ürünüm Gelmedi!,Sizden 14/05/2020 tar...
1  alisveris  Teknosa Preo Mouse Driver Sorunu,Ben yaklaşık ...
2  alisveris  Vatan Bilgisayar Ücret İadesi Sorunu,"08 Mayıs...
3  alisveris  Vatan Bilgisayar Hala Kargoya Verilmemesi,"29 ...
4  alisveris  Migros'ta  Kıyma Çok Kötü,Yarım kg kıyma aldık...


In [30]:
import re

def clean_text(text):
    # Özel karakterleri, sembolleri ve sayıları temizle
    text = re.sub(r"[^a-zA-ZüöçğışıĞÜÖÇİŞ ]", "", text)
    # Birden fazla boşluğu tek boşluğa indir
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Metinleri temizle
data['cleaned_text'] = data['text'].apply(clean_text)


In [31]:
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# Türkçe stop word listesi
turkish_stopwords = set(stopwords.words("turkish"))

def process_text(text):
    sentences = sent_tokenize(text)  # Metni cümlelere bölelim
    
    # Her cümleyi kelimelere bölelim ve stop words'leri çıkaralım
    processed_sentences = []
    for sentence in sentences:
        words = word_tokenize(sentence)
        filtered_words = [word for word in words if word.lower() not in turkish_stopwords]
        processed_sentences.append(filtered_words)
    
    return processed_sentences

# Cümleleri ve stop words'leri işle
data['processed_text'] = data['cleaned_text'].apply(process_text)


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF vektörleştiriciyi oluştur
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # En fazla 1000 kelime kullanacağız

# İşlenmiş cümleleri birleştirerek metinlere dönüştür
corpus = [' '.join(words) for words in data['processed_text'].apply(lambda words: [' '.join(sentence) for sentence in words])]

# TF-IDF matrisini oluştur
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

# TF-IDF matrisini DataFrame'e çevir
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Sonuçları incele
print(tfidf_df.head())



   acil  acilen  ade     adesi  adet  adevamını  adres  adı  adına  aile  ...  \
0   0.0     0.0  0.0  0.000000   0.0        0.0    0.0  0.0    0.0   0.0  ...   
1   0.0     0.0  0.0  0.000000   0.0        0.0    0.0  0.0    0.0   0.0  ...   
2   0.0     0.0  0.0  0.229632   0.0        0.0    0.0  0.0    0.0   0.0  ...   
3   0.0     0.0  0.0  0.000000   0.0        0.0    0.0  0.0    0.0   0.0  ...   
4   0.0     0.0  0.0  0.000000   0.0        0.0    0.0  0.0    0.0   0.0  ...   

   şirket  şirketi  şok  şubat  şube  şubesi  şubesinde  şubesinden  şubesine  \
0     0.0      0.0  0.0    0.0   0.0     0.0        0.0         0.0       0.0   
1     0.0      0.0  0.0    0.0   0.0     0.0        0.0         0.0       0.0   
2     0.0      0.0  0.0    0.0   0.0     0.0        0.0         0.0       0.0   
3     0.0      0.0  0.0    0.0   0.0     0.0        0.0         0.0       0.0   
4     0.0      0.0  0.0    0.0   0.0     0.0        0.0         0.0       0.0   

   şubeye  
0     0.0  
1 

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

# Cümle benzerlik matrisini oluştur
sentence_similarity_matrix = cosine_similarity(tfidf_matrix)

# Sonuçları incele (örneğin ilk 5 cümle arası benzerlik)
print(sentence_similarity_matrix[:5, :5])


[[1.         0.03662926 0.04794504 0.08134647 0.04771625]
 [0.03662926 1.         0.02268624 0.02091655 0.00314593]
 [0.04794504 0.02268624 1.         0.19991972 0.        ]
 [0.08134647 0.02091655 0.19991972 1.         0.        ]
 [0.04771625 0.00314593 0.         0.         1.        ]]


In [35]:
from sklearn.metrics.pairwise import cosine_similarity

# Kelime benzerlik matrisini oluştur (sütunları transpoz yaparak kullanıyoruz)
word_similarity_matrix = cosine_similarity(tfidf_matrix.T)

# Sonuçları incele (örneğin ilk 5 kelime arası benzerlik)
print(word_similarity_matrix[:5, :5])


[[1.         0.02167297 0.02019878 0.02305615 0.0020506 ]
 [0.02167297 1.         0.02119619 0.00905196 0.01524126]
 [0.02019878 0.02119619 1.         0.01932495 0.03948261]
 [0.02305615 0.00905196 0.01932495 1.         0.02141202]
 [0.0020506  0.01524126 0.03948261 0.02141202 1.        ]]


In [ ]:
import networkx as nx
import numpy as np

# Cümle benzerlik matrisini kullanarak bir graf oluştur
def create_sentence_graph(similarity_matrix):
    num_sentences = similarity_matrix.shape[0]
    graph = nx.Graph()

    # Düğümleri ekle
    graph.add_nodes_from(range(num_sentences))

    # Kenarları ekle
    for i in range(num_sentences):
        for j in range(i + 1, num_sentences):
            similarity = similarity_matrix[i, j]
            graph.add_edge(i, j, weight=similarity)

    return graph

# Kelime benzerlik matrisini kullanarak bir graf oluştur
def create_word_graph(similarity_matrix, words):
    num_words = similarity_matrix.shape[0]
    graph = nx.Graph()

    # Düğümleri ekle
    graph.add_nodes_from(range(num_words))

    # Kelimeleri düğümlere eşle
    node_labels = {i: word for i, word in enumerate(words)}
    graph = nx.relabel_nodes(graph, node_labels)

    # Kenarları ekle
    for i in range(num_words):
        for j in range(i + 1, num_words):
            similarity = similarity_matrix[i, j]
            graph.add_edge(words[i], words[j], weight=similarity)

    return graph

# Örnek graf oluşturma (cümle benzerlik matrisi kullanarak)
sentence_graph = create_sentence_graph(sentence_similarity_matrix)
# Örnek graf oluşturma (kelime benzerlik matrisi kullanarak)
word_graph = create_word_graph(word_similarity_matrix, tfidf_vectorizer.get_feature_names_out())

# Grafın düğüm sayısı ve kenar sayısı
print("Sentence Graph Nodes:", sentence_graph.number_of_nodes())
print("Sentence Graph Edges:", sentence_graph.number_of_edges())
print("Word Graph Nodes:", word_graph.number_of_nodes())
print("Word Graph Edges:", word_graph.number_of_edges())
